# Time Series Analysis

## Reading data

In [1]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [2]:
df = pd.read_csv("../data/processed/forecasting_data.csv")
df["DATA"] = pd.to_datetime(df["DATA"])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5033 entries, 0 to 5032
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   DATA               5033 non-null   datetime64[ns]
 1   VALOR_REEMBOLSADO  5033 non-null   float64       
 2   COUNT              5033 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 118.1 KB


In [3]:
# Check if all years have 365 days
df.groupby(df["DATA"].dt.year)["DATA"].count()

DATA
2009    285
2010    365
2011    365
2012    366
2013    365
2014    365
2015    365
2016    366
2017    365
2018    365
2019    365
2020    366
2021    365
2022    365
Name: DATA, dtype: int64

## Fixing year 2009

We can see above that in 2009, some dates are missing to compose the time series. To solve this, we need to find which dates are missing and define an approach.


In [4]:
# Find the missing days of 2009
missing_days = pd.date_range(start="2009-01-01", end="2009-12-31").difference(
    df[df["DATA"].dt.year == 2009]["DATA"]
)
missing_days

DatetimeIndex(['2009-01-01', '2009-01-02', '2009-01-03', '2009-01-04',
               '2009-01-05', '2009-01-06', '2009-01-07', '2009-01-08',
               '2009-01-09', '2009-01-10', '2009-01-11', '2009-01-12',
               '2009-01-13', '2009-01-14', '2009-01-15', '2009-01-16',
               '2009-01-17', '2009-01-18', '2009-01-19', '2009-01-20',
               '2009-01-21', '2009-01-22', '2009-01-23', '2009-01-24',
               '2009-01-25', '2009-01-27', '2009-01-28', '2009-01-29',
               '2009-01-30', '2009-01-31', '2009-02-01', '2009-02-02',
               '2009-02-03', '2009-02-04', '2009-02-05', '2009-02-06',
               '2009-02-07', '2009-02-08', '2009-02-09', '2009-02-10',
               '2009-02-11', '2009-02-12', '2009-02-13', '2009-02-14',
               '2009-02-15', '2009-02-16', '2009-02-17', '2009-02-18',
               '2009-02-19', '2009-02-20', '2009-02-21', '2009-02-22',
               '2009-02-23', '2009-02-24', '2009-02-25', '2009-02-26',
      

We can see above that up intil April, most of the dates are missing. Since the missing dates are centered in those first months, the approach defined was to remove the few dates from these months and begin the times series from April.

In [5]:
# Drop the rows where the data is less than 2009-04-01
df = df[df["DATA"] >= "2009-04-01"]

## Ploting the data

In [6]:
# Chart default layout
default_layout = dict(
    titlefont=dict(size=18, color="darkblue"),
    tickfont=dict(size=14, color="black"),
    showgrid=True,
    zeroline=True,
    showline=True,
    mirror=True,
    gridcolor="lightgrey",
    gridwidth=1,
    zerolinecolor="grey",
    zerolinewidth=2,
    linecolor="black",
    linewidth=2,
)

In [7]:
# Define x and y axis layout
xaxis_layout = default_layout.copy()
xaxis_layout["title"] = "Data"

yaxis_layout = default_layout.copy()
yaxis_layout["title"] = "Valor reembolsado (R$)"

In [8]:
# Calculate rolling mean, variance and standard deviation with
# a window size of 7 (3 previous + current + 3 next)
window_size = 7
df["Rolling_Mean"] = (
    df["VALOR_REEMBOLSADO"].rolling(window=window_size, center=True).mean()
)
df["Rolling_Variance"] = (
    df["VALOR_REEMBOLSADO"].rolling(window=window_size, center=True).var()
)
df["Rolling_Std_Dev"] = np.sqrt(df["Rolling_Variance"])

In [9]:
fig = go.Figure()

# Add a shaded area for the variance (optional, showing ± one standard deviation)
fig.add_trace(
    go.Scatter(
        x=np.concatenate([df["DATA"], df["DATA"][::-1]]),
        y=np.concatenate(
            [
                (df["Rolling_Mean"] + df["Rolling_Std_Dev"]).fillna(0),
                (df["Rolling_Mean"] - df["Rolling_Std_Dev"]).fillna(0)[::-1],
            ]
        ),
        fill="toself",
        fillcolor="rgba(253, 143, 143, 0.5)",  # Light blue with transparency
        line=dict(color="rgba(255, 255, 255, 0.0)"),
        name="Variância móvel (±1 std dev)",
    )
)

# Add the main timeseries values
fig.add_trace(
    go.Scatter(
        x=df["DATA"],
        y=df["VALOR_REEMBOLSADO"],
        mode="lines",
        marker=dict(size=10, color="blue", symbol="circle"),
        line=dict(width=2, color="blue"),
        name="Valor real reembolsado",
    )
)

# Add a horizontal line for the rolling mean
fig.add_trace(
    go.Scatter(
        x=df["DATA"],
        y=df["Rolling_Mean"],
        mode="lines",
        name="Média móvel (3 dias anteriores + atual + 3 dias seguintes)",
        line=dict(color="red"),
    )
)

# Set chart layout
layout = go.Layout(
    title="Reimbursement by Year",
    titlefont=dict(size=24, color="darkblue"),
    xaxis=xaxis_layout,
    yaxis=yaxis_layout,
    hovermode="closest",
    plot_bgcolor="white",
)

fig.show()
# py.iplot({"data": [trace], "layout": layout})

## Checking stationarity of the time series

A time series is stationary when its statistical properties, like its mean and variance, do not change over time.
![stationary series](https://miro.medium.com/v2/resize:fit:720/format:webp/0*Dyml4bSlkE5WHdcc)

One way to check if a time series is stationary is by the [Dickey-Fuller Test](https://medium.com/@ritusantra/tests-for-stationarity-in-time-series-dickey-fuller-test-augmented-dickey-fuller-adf-test-d2e92e214360). Basically, we need that the ADF statistic to be less than the critical value of 1%.

In [10]:
from statsmodels.tsa.stattools import adfuller

In [11]:
X = df["VALOR_REEMBOLSADO"]
result = adfuller(X)
print("p-value: %f" % result[1])
print("ADF Statistic: %f" % result[0])
print("Critical Values:")
for key, value in result[4].items():
    print("\t%s: %.3f" % (key, value))

p-value: 0.000002
ADF Statistic: -5.463028
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567


From the results above, we can conclude:
- `p-value` is equal to 0.000002: Due to the low value (p-value <= 0.05), it suggests that the timeseries might be stationary.
- `ADF Statistic` equal to -5.463: The more negative this statistic, the more likely to be stationary.
-  The `ADF Statistic` is below the `critical value of 1%`: This suggests that we can reject the null hypothesis (the existence of any trend) with a significance level of less than 1% (i.e. a low probability that the result has any trend, concluding the timeseries is stationary).

In conclusion, we can use the ADF Test to verify the stationarity of timeseries data. We need to verify the following conditions:
- `p-value <= threshold`: generally the threshold is equal to 0.05
- `adf-statistic < 0`: if it is negative
- `adf-statistic < critical-value-1`: if the adf statistic is below the critical value of 1% calculated by the ADF test.

If these conditions are satisfied, we can assure in some way that the timeseries is stationary.

In [12]:
def check_stationarity(
    series: pd.Series, threshold: float = 0.05, print_results: bool = False
):
    """
    Check the stationarity of a timeseries data.
    :param series: Time Series data
    :param threshold: Threshold for comparing p-value
    :param print_results: Indicates if the results must be printed
    :return: Bool indicating if the data is stationary
    """
    stationarity: bool = False
    result = adfuller(series)

    if result[1] <= threshold and result[0] < 0 and result[4]["1%"] < [4]:
        stationarity = True

    if print_results:
        print("p-value: %f" % result[1])
        print("ADF Statistic: %f" % result[0])
        print("Critical Values:")
        for key, value in result[4].items():
            print("\t%s: %.3f" % (key, value))
        print(f"The TimeSeries is {'not ' if not stationarity else ''}stationary.")

    return stationarity

In [13]:
res = check_stationarity(df["VALOR_REEMBOLSADO"], print_results=True)

p-value: 0.000002
ADF Statistic: -5.463028
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
The TimeSeries is stationary.


## Treating stationarity of the time series (if it was required)

there are several common techniques you can use to transform it into a stationary series. These techniques aim to remove trends, seasonality, or other time-dependent structures. Here are some methods:
- **First differencing** or **seasonal differencing**
- **Log transformation**
- **Square root** or **Cube root** transformation
- Smoothing transformations, decomposition, and others...

### First differencing or seasonal differencing

Differencing is one of the simplest and most commonly used methods to make a time series stationary.

On **first differencing**, we subtract the current value from the previous value to remove trends.

If your data has seasonal patterns (e.g., monthly data with yearly seasonality), we subtract the value from the same period in the previous season (**seasonal differencing**). 

In [14]:
# Set the periods equals to the seasonality
# df['VALOR_REEMBOLSADO_stationary'] = df['VALOR_REEMBOLSADO'].diff(periods=1)

Differencing can be repeated if necessary (e.g., second-order differencing), but be careful not to over-difference, as this can make the series overly complex and difficult to model.

### Log transformation

A log transformation can help stabilize the variance, especially if your data has an exponential trend.

In [15]:
# df['VALOR_REEMBOLSADO_stationary'] = np.log(df['VALOR_REEMBOLSADO'])

### Square root of Cube root transformations

If a log transformation is not suitable, you can try square root or cube root transformations to stabilize variance. These transformations are less aggressive than a log transformation.

In [16]:
# Square root transformation
# df['VALOR_REEMBOLSADO_stationary'] = np.sqrt(df['VALOR_REEMBOLSADO'])

In [17]:
# Cube root transformation
# df['VALOR_REEMBOLSADO_stationary'] = np.cbrt(df['VALOR_REEMBOLSADO'])